In [ ]:
# TODO MAKE The connections to all but with a size differnce

import IPython.display

import matplotlib.pyplot as plt

import imageio.v2 as imageio
import os
from datetime import datetime
from machine.mlp import MyModel
import matplotlib.animation as animation

# Set up the file paths
curpath = r'C:\Users\abelt\OneDrive\Dokumenter\GitHub\Ship_datafusion\First_step_mathching'
folderpath = os.path.join(curpath, 'frames')
os.makedirs(folderpath, exist_ok=True)

# Clear folder for frames
def clear_folder(folder = folderpath):
    os.makedirs(folderpath, exist_ok=True)
    # Directory for saving frames, and cleaning it every run
    for item in os.listdir(folder):
        os.remove(os.path.join(folderpath, item))





In [ ]:
# Forward with heading
def forward(t1, t2, model):
    e1 = model(torch.tensor(t1, dtype=torch.float32))
    e2 = model(torch.tensor(t2, dtype=torch.float32))
    return torch.cdist(e1,e2)

In [ ]:
# PLAY AREA:


In [ ]:
if __name__ == '__main__':
    # Set up model and optimizer
    maxiter = 300
    N = 250s
    N_test = 50
    sigma_distance = 0.025
    sigma_heading = 0.005
    encode = True
    missing_percentage = 0.6

    ranks = [1, 3, 5]

    mlp_params = {
        'input_dim': 4,
        'hidden_dim': 1024,
        'output_dim': 254,
        'n_layers': 5,
        'dropout': 0.2,
        'learning_rate': 0.00001,
    }

    # Directory for saving frames, and cleaning it every run
    curpath = r'C:\Users\abelt\OneDrive\Dokumenter\GitHub\Ship_datafusion\First_step_mathching'
    folderpath = os.path.join(curpath, 'frames')
    
    clear_folder(folder=folderpath)

    # Initialize variables
    frames = []
    losses = {'train': [], 'val': []}

    # Create empty lists to store accuracies
    rank_accuracies = {rank: [] for rank in ranks}

    # Generate validation data
    p1, p2 = generate_data(N_test, sigma=sigma_distance)
    t1, t2 = calculate_heading(p1,p2, encoding=encode, missing_percentage=missing_percentage)
    
    model = MyModel(input_dim=mlp_params['input_dim'], hidden_dim=mlp_params['hidden_dim'], 
                output_dim=mlp_params['output_dim'], depth=mlp_params['n_layers'], 
                drop_prob=mlp_params['dropout'])
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=mlp_params['learning_rate'])
    # Training loop
    for i in range(maxiter + 1):
        model.train()
        optimizer.zero_grad()
        p1t, p2t = generate_data(N, sigma=sigma_distance)
        t1p, t2p = calculate_heading(p1t,p2t, encoding=encode, missing_percentage=missing_percentage)
        
        diag_matrix = forward(t1p, t2p, model)
        
        # Calculate training loss and backpropagate
        diag = torch.diag(diag_matrix)
        train_loss = torch.sum(diag)
        train_loss.backward()
        losses['train'].append(train_loss.item())
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            #diag_matrix_val = forward(p1, p2, model)
            diag_matrix_val = forward(t1, t2, model)

            diag_val = torch.diag(diag_matrix_val)
            val_loss = torch.sum(diag_val)
            losses['val'].append(val_loss.item())

        # Plot and save frame every 10 iterations
        if i % 10 == 0 or i == 1: # and i > 0
            # Rank-N evaluation
            for rank in ranks:
                accuracy = evaluate_rankN(diag_matrix, rank)
                rank_accuracies[rank].append(accuracy.item())

            d = torch.argmin(diag_matrix_val, dim=1).detach().numpy()

            # Plotting
            fig, (ax1, ax2, ax3) = plot_iteration_with_loss_and_accuracy(
                t1, t2, diag_matrix_val, np.arange(N_test), losses, rank_accuracies, train_loss.item(), i, maxiter
            )
            frame_path = os.path.join(folderpath, f'frame_{i}.png')
            fig.savefig(frame_path)
            frames.append(imageio.imread(frame_path))
            plt.close(fig)

    # Save the video
    imageio.mimsave(os.path.join(folderpath, '0_training_epochs.mp4'), frames, fps=3)

TODO:
- Lav det med AIS data og SAR data som "Test data".
- Lav en tabel hvor der ændres på forskellige parametre: sigma, N og N_features (Her skal cog og sog regnes med)
- Tilføj cog og sog i generatoren

In [ ]:
def plot_with_headings(p1, p2, headings):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_ylim(0, 1)
    ax.set_xlim(0, 1)
    # Plot the points
    ax.scatter(p1[0], p1[1], c='b', label="Track 1", s=5)
    ax.scatter(p2[0], p2[1], c='r', label="Track 2", s=5)
    
    # Plot connections
    for i in range(len(p1[0])):
        ax.plot([p1[0][i], p2[0][i]], [p1[1][i], p2[1][i]], 'g-')

    # Plot arrows to indicate heading
    dx = np.cos(headings)*0.05  # x component of the direction
    dy = np.sin(headings)*0.05 # y component of the direction
    ax.quiver(p1[0], p1[1], dx, dy, angles='xy', scale_units='xy', scale=1, color='purple', width=0.003)

    # Add legend and display the plot
    ax.legend(loc="upper left")
    plt.show()

# Example usage
N = 5
sigma = 0.075
p1, p2 = generate_data(N, sigma)
h1, h2 = calculate_heading(p1, p2, degrees=False, sigma=0.05)
plot_with_headings(p1, p2, h1)

In [ ]:
mean_heading = 0  # in degrees, representing the direction (0,0) to (1,0)
std_dev = 15  # standard deviation in degrees
num_samples = 1000  # number of samples to simulate

# Generate heading samples from a normal distribution centered at 0 degrees
headings = np.random.normal(mean_heading, std_dev, num_samples)
# Adjust heading values to wrap around the unit circle (0 to 360 degrees)
headings = np.mod(headings, 360)
# Generate x and y coordinates based on the heading angles on the unit circle
x_coords = np.cos(np.radians(headings))
y_coords = np.sin(np.radians(headings))



# Plot the histogram of the adjusted heading distribution
plt.figure(figsize=(8, 6))
plt.hist(headings, bins=30, color='lightcoral', edgecolor='black', density=True)
plt.xlabel("Heading (degrees)")
plt.ylabel("Probability Density")
plt.title("Simulated Heading Distribution Aligned with Direction (0,0) to (1,0)")
plt.grid(True)
plt.show()

# Plot the unit circle with simulated heading vectors
plt.figure(figsize=(8, 8))
# Plot the unit circle boundary
theta = np.linspace(0, 2 * np.pi, 1000) 
plt.plot(np.cos(theta), np.sin(theta), label='Unit Circle')

# Plot the heading vectors as points on the unit circle
plt.scatter(x_coords, y_coords, color='lightcoral',marker='o', alpha=0.7, s=30, label='Simulated Headings')

# Add axes and labels
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Simulated Heading Distribution on Unit Circle')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
# experiment
# # -*- coding: utf-8 -*-
# import numpy as np
# import matplotlib.pyplot as plt
# import torch
# import imageio.v2 as imageio
# import os
# from datetime import datetime
# from machine.mlp import MyModel
# import matplotlib.animation as animation

# # Normalize the data (optional)
# def normalize(data):
#     min_val = np.min(data)
#     max_val = np.max(data)
#     return (data - min_val) / (max_val - min_val)

# # Generate synthetic data
# def generate_data(N, sigma):
#     np.random.seed(1)
#     g = np.random.Generator(np.random.PCG64())
#     x = g.uniform(0, 1, N)
#     y = g.uniform(0, 1, N)
#     x1 = np.clip(g.normal(x, sigma), a_min=0, a_max=1)  
#     y1 = np.clip(g.normal(y, sigma), a_min=0, a_max=1)
#     track1 = np.stack((x, y))
#     track2 = np.stack((x1, y1))
#     return track1, track2

# # Plot points
# def plot_tracks(track1, track2):
#     fig, ax = plt.subplots(dpi=100)
#     ax.scatter(track1[0], track1[1], c='b', marker='o', label="Track 1")
#     ax.scatter(track2[0], track2[1], c='r', marker='x', label="Track 2")
#     ax.legend()
#     return fig, ax

# # Plot connections with dynamic line width

# def plot_connection(ax, track1, track2, assignment, **kwargs):
#     for i, j in enumerate(assignment):
#         t1 = track1.T[i]
#         t2 = track2.T[j]
#         #dist = np.linalg.norm(t1 - t2)
#         ax.plot([t1[0], t2[0]], [t1[1], t2[1]],  **kwargs)#linewidth=2.5/(dist + 1e-5),

# def plot_loss(losses):
#     fig, ax = plt.subplots()
#     ax.plot(losses)
#     ax.set_title("Training Loss Over Iterations")
#     ax.set_xlabel("Iterations")
#     ax.set_ylabel("Loss")

#     plt.show()
#     return fig, ax

# # Set up model and optimizer
# mlp_params = {
#     'input_dim': 2,
#     'hidden_dim': 1024,
#     'output_dim': 254,
#     'n_layers': 2,
#     'dropout': 0.1,
# }
# model = MyModel(input_dim=mlp_params['input_dim'], hidden_dim=mlp_params['hidden_dim'], 
#                 output_dim=mlp_params['output_dim'], depth=mlp_params['n_layers'], 
#                 drop_prob=mlp_params['dropout'])

# model.train()
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

# # Directory for saving frames, and cleaning it every run
# curpath = r'C:\Users\abelt\OneDrive\Dokumenter\GitHub\Ship_datafusion\First_step_mathching'
# folderpath = os.path.join(curpath, 'frames')
# os.makedirs(folderpath, exist_ok=True)
# for item in os.listdir(folderpath):
#     os.remove(os.path.join(folderpath, item))

# # Initialize variables
# frames = []
# losses = []
# N = 25
# sigma = 0.05
# track1, track2 = generate_data(N, sigma)

# # Training loop
# for i in range(10000):
#     optimizer.zero_grad()
#     t1 = torch.tensor(track1.T, dtype=torch.float32)
#     t2 = torch.tensor(track2.T, dtype=torch.float32)
#     e1 = model(t1)
#     e2 = model(t2)

#     diag_matrix = torch.cdist(e1, e2)
#     diag = torch.diag(diag_matrix)
#     loss = torch.sum(diag)
#     loss.backward()
#     losses.append(loss.item())
#     optimizer.step()
    
#     # Plot and save frame every 10 iterations
#     if i % 10 == 0:
#         fig, ax = plot_tracks(track1, track2)
#         plot_connection(ax, track1, track2, np.arange(N), color='g')
#         d = torch.argmin(diag_matrix, dim=1).detach().numpy()
#         plot_connection(ax, track1, track2, d, color='r')
    
#         ax.set_title(f"Iteration: {i}")

#         frame_path = os.path.join(folderpath, f'frame_{i}.png')
#         fig.savefig(frame_path)
#         frames.append(imageio.imread(frame_path))
#         plt.close(fig)


# # Save the video
# imageio.mimsave(os.path.join(folderpath, '0_training_epochs.mp4'), frames, fps=5)

# def plot_loss(losses):
#     fig, ax = plt.subplots()
#     ax.plot(losses)
#     ax.set_title("Training Loss Over Iterations")
#     ax.set_xlabel("Iterations")
#     ax.set_ylabel("Loss")

#     plt.show()
#     return fig, ax
    
# # Plot the loss curve
# plot_loss(losses)



# def plot_iteration_with_loss_and_accuracy(p1, p2, diag_matrix, green_indices, losses, rank_accuracies, loss, iteration, max_iter):
#     """
#     Creates a subplot with the track plot on top, the loss curve in the bottom left, and the rank-N accuracy curve in the bottom right.

#     Parameters:
#         p1 (ndarray): Points for the first track, shape (2, N).
#         p2 (ndarray): Points for the second track, shape (2, N).
#         diag_matrix (ndarray): Diagonal matrix of the distance matrix.
#         green_indices (array-like): Indices for green connections (e.g., correct matches).
#         losses (list of float): List of loss values over training iterations.
#         rank_accuracies (dict of lists): Dictionary where keys are rank values and values are lists of accuracy values over iterations.
#         loss (float): Loss value for the current iteration.
#         iteration (int): Current iteration number.
#         max_iter (int): Maximum number of iterations.
        
#     Returns:
#         fig (Figure): The matplotlib figure object containing all three subplots.
#         (ax1, ax2, ax3): Tuple of axes objects for the track, loss, and accuracy plots.
#     """

#     # Create a figure and a GridSpec layout for custom positioning
#     fig = plt.figure(figsize=(12, 8), dpi=100)
#     gs = fig.add_gridspec(2, 2, height_ratios=[3, 1], hspace=0.4, wspace=0.3)
    
#     # Top plot: Track connections (spanning both columns)
#     ax1 = fig.add_subplot(gs[0, :])
#     ax1.scatter(p1[0], p1[1], c='b', marker='o', label="Track 1")
#     ax1.scatter(p2[0], p2[1], c='r', marker='x', label="Track 2")
#     # Actual connection
#     plot_connection(ax1, p1, p2, green_indices, color='g')
    
#     # MLP connection
#     d = torch.argmin(diag_matrix, dim=1).detach().numpy()
#     plot_connection(ax1, p1, p2, d, color='r')
#     # Heading
#     plot_direction_vectors(ax1, p1, p2, color1='blue', color2='red', width=0.003)
#     ax1.set_title(f"Iteration: {iteration}, Loss: {loss:.4f}")
#     ax1.legend(loc='upper left', bbox_to_anchor=(1.009, 1), fancybox=True, shadow=True)

#     # Bottom left plot: Loss curve
#     ax2 = fig.add_subplot(gs[1, 0])
#     #ax2.plot(losses, label="Training Loss", color='b')
#     ax2.plot(losses['train'], label="Training Loss", color='b')
#     ax2.plot(losses['val'], label="Validation Loss", color='orange', linestyle='--')
#     ax2.set_xlim(0, max_iter + 1)
#     ax2.set_title("Training Loss Over Iterations")
#     ax2.set_xlabel("Iterations")
#     ax2.set_ylabel("Loss")
#     ax2.legend(loc='upper right', fancybox=True, shadow=True)

#     # Bottom right plot: Rank accuracy curves
#     ax3 = fig.add_subplot(gs[1, 1])
#     for rank, accuracies in rank_accuracies.items():
#         ax3.plot(range(len(accuracies)), accuracies, label=f'Rank {rank}',  marker='o', linestyle='dashed')
#     ax3.set_xlim(0, (max_iter + 1) / 10)
#     ax3.set_ylim(0, 1)
#     ax3.set_title("Rank-N Accuracy Over Iterations")
#     ax3.set_xlabel("Iterations, [i/10]")
#     ax3.set_ylabel("Accuracy")
#     ax3.legend(loc='upper left', bbox_to_anchor=(1.02, 1), fancybox=True, shadow=True)
    
#     return fig, (ax1, ax2, ax3)